In [129]:
import pandas as pd

df = pd.read_csv('cleaned_vers_data_covid19_vaccine_2020-2022_duplicate_removed.csv', encoding='latin-1')


C:\Users\rutub\AppData\Local\Temp\ipykernel_13528\2592829857.py:3: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_vers_data_covid19_vaccine_2020-2022_duplicate_removed.csv', encoding='latin-1')


In [130]:
# Extract the specified columns
columns = ['VAERS_ID', 'AGE_YRS', 'SEX', 'DIED', 'L_THREAT', 'HOSPITAL', 'DISABLE', 'OFC_VISIT', 'ER_ED_VISIT', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'ALLERGIES', 'VAX_MANU', 'Symptoms']
df = df[columns]

# Display the extracted columns
df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,L_THREAT,HOSPITAL,DISABLE,OFC_VISIT,ER_ED_VISIT,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,Symptoms
0,902418,56.0,F,N,N,N,N,N,N,latex,none,none,none,PFIZER\BIONTECH,"Hypoaesthesia, Injection site hypoaesthesia"
1,902440,35.0,F,N,N,N,N,N,N,No medication,Not applicable,No concerns,N,PFIZER\BIONTECH,Headache
2,902446,55.0,F,N,N,N,N,N,N,"Novasc, Hydrochlorothiazide, synthroid, lisino...",none,"Hypertension, sleep apnea, hypothyroidism","Contrast Dye IV contrast, shellfish, strawberry",PFIZER\BIONTECH,"Erythema, Feeling hot, Flushing"
3,902464,42.0,M,N,N,N,N,N,Y,none,none,none,none,PFIZER\BIONTECH,"Dizziness, Electrocardiogram normal, Hyperhidr..."
4,902465,60.0,F,N,N,N,N,N,N,"Lisinopril, HCTZ, lipitor, amyitrypline, lexap...","Bronchitis, finished prednisone on 12-13-20","hypertension, fibromyalgia",Biaxin,PFIZER\BIONTECH,"Dysgeusia, Sensory disturbance, Oral pruritus,..."


In [131]:
def map_age_group(age):
    if age >= 0 and age <= 10:
        return 'AG1'
    elif age >= 11 and age <= 20:
        return 'AG2'
    elif age >= 21 and age <= 30:
        return 'AG3'
    elif age >= 31 and age <= 40:
        return 'AG4'
    elif age >= 41 and age <= 50:
        return 'AG5'
    elif age >= 51 and age <= 60:
        return 'AG6'
    elif age >= 61 and age <= 70:
        return 'AG7'
    elif age >= 71 and age <= 80:
        return 'AG8'
    elif age >= 81 and age <= 90:
        return 'AG9'
    elif age >= 91 and age <= 100:
        return 'AG10'
    else:
        return 'AGN'  # Default age group for values outside the specified ranges

df['AGE_GROUP'] = df['AGE_YRS'].apply(map_age_group)
df = df.drop('AGE_YRS', axis=1)

In [132]:
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

df.loc[df['OTHER_MEDS'].str.startswith(('none', 'None', 'NONE', 'NOS', 'not', 'no', 'Not', 'na', 'NO', 'NOT', 'unknown', 'Unknown')), 'OTHER_MEDS'] = 'no medication'
df.loc[df['CUR_ILL'].str.startswith(('none', 'None', 'NONE', 'NOS', 'not', 'no', 'Not', 'na', 'NO', 'NOT', 'No', 'negative', 'non', 'Non', 'Nothing', 'nothing', 'unknown', 'Unknown')), 'CUR_ILL'] = 'not applicable'
df.loc[df['HISTORY'].str.startswith(('none', 'None', 'NONE', 'NOS', 'not', 'no', 'Not', 'na', 'NO', 'NOT', 'No', 'negative', 'non', 'Non', 'Nothing', 'nothing', 'unknown', 'Unknown')), 'HISTORY'] = 'no concerns'
df.loc[df['ALLERGIES'].str.startswith(('none', 'None', 'NONE', 'NOS', 'not', 'no', 'Not', 'na', 'NO', 'NOT', 'No', 'negative', 'non', 'Non', 'Nothing', 'nothing', 'nka', 'nkda', 'NKA', 'NKDA', 'unknown', 'Unknown', 'n')), 'ALLERGIES'] = 'no allergies'

In [133]:
pfizer_biontech_df = df[df['VAX_MANU'] == 'pfizer\\biontech']
unknown_manufacturer_df = df[df['VAX_MANU'] == 'unknown manufacturer']
moderna_df = df[df['VAX_MANU'] == 'moderna']
janssen_df = df[df['VAX_MANU'] == 'janssen']

In [134]:
pfizer_biontech_df.head()
num_rows = pfizer_biontech_df.shape[0]
print("Number of rows in pfizer_biontech_df:", num_rows)


Number of rows in pfizer_biontech_df: 232669


In [135]:
moderna_df.head()
num_rows = moderna_df.shape[0]
print("Number of rows in moderna_df:", num_rows)

Number of rows in moderna_df: 282089


In [136]:
unknown_manufacturer_df.head()
num_rows = unknown_manufacturer_df.shape[0]
print("Number of rows in unknown_manufacturer_df:", num_rows)

Number of rows in unknown_manufacturer_df: 937


In [137]:
janssen_df.head()
num_rows = janssen_df.shape[0]
print("Number of rows in janssen_df:", num_rows)

Number of rows in janssen_df: 41034


In [138]:
def categorize_symptoms(symptom):
    categories = {
        'pain': ['headache', 'pain', 'pain in extremity', 'chest discomfort', 'chest pain', 'abdominal pain upper', 'pain of skin', 'abdominal pain', 'bone pain', 'back pain', 'neck pain', 'musculoskeletal pain', 'arthralgia', 'myalgia', 'joint swelling', 'muscle spasms', 'muscle tightness', 'muscular weakness', 'joint range of motion decreased', 'musculoskeletal stiffness', 'joint stiffness', 'neck stif'],
        'fever/chills': ['pyrexia', 'chills', 'body temperature increased', 'feeling hot', 'feeling cold', 'cold sweat', 'night sweats', 'hyperhidrosis', 'fever', 'influenza', 'influenza like illness', 'covid-19', 'febrile neutropenia', 'feeling abnormal', 'feeling hot and cold', 'feeling cold and hot'],
        'fatigue/general discomfort': ['fatigue', 'asthenia', 'malaise', 'lethargy', 'condition aggravated', 'discomfort',],
        'skin reactions': ['injection site erythema', 'pruritus', 'injection site pruritus', 'rash', 'erythema', 'rash erythematous', 'rash pruritic', 'hyperhidrosis', 'skin warm', 'skin swelling', 'urticaria', 'skin discolouration', 'skin burning sensation', 'skin lesion', 'skin exfoliation', 'skin induration', 'skin tightness', 'skin irritation', 'skin reaction', 'skin ulcer', 'skin hypertrophy', 'skin atrophy', 'skin nodule', 'skin papilloma', 'skin hyperpigmentation', 'skin hypopigmentation', 'skin haemorrhage', 'skin necrosis', 'skin striae', 'skin wrinkling', 'skin fragility', 'skin exfoliation', 'skin depigmentation'],
        'digestive issues': ['nausea', 'vomiting', 'diarrhoea', 'abdominal discomfort', 'dysphagia', 'retching', 'abdominal distension', 'abdominal pain', 'abdominal pain upper', 'abdominal pain lower', 'abdominal tenderness', 'abdominal rigidity', 'abdominal pain right', 'abdominal pain left', 'abdominal pain upper', 'abdominal pain lower', 'abdominal tenderness', 'abdominal rigidity', 'abdominal pain right', 'abdominal pain left', 'abdominal pain upper', 'abdominal pain lower', 'abdominal tenderness', 'abdominal rigidity', 'abdominal pain right'],
        'respiratory symptoms': ['dyspnoea', 'cough', 'rhinorrhoea', 'nasal congestion', 'respiratory tract congestion', 'wheezing', 'dysphonia', 'respiratory failure', 'respiratory distress', 'respiratory arrest', 'respiratory depression', 'respiratory disorder', 'respiratory rate increased', 'respiratory tract infection', 'respiratory disorder', ],
        'neurological symptoms': ['dizziness', 'paraesthesia', 'hypoaesthesia', 'tremor', 'syncope', 'vertigo', 'seizure', 'loss of consciousness', 'confusional state', 'disorientation', 'presyncope'],
        'cardiovascular symptoms': ['palpitations', 'heart rate increased', 'hypertension', 'hypotension', 'tachycardia', 'chest discomfort'],
        'musculoskeletal symptoms': ['arthralgia', 'myalgia', 'muscle spasms', 'muscular weakness', 'joint swelling', 'musculoskeletal stiffness', 'joint range of motion decreased', 'muscle tightness'],
        'psychiatric symptoms': ['anxiety', 'depression', 'nervousness', 'insomnia', 'somnolence', 'hallucination', 'psychosis'],
        'visual and auditory symptoms': ['tinnitus', 'vision blurred', 'eye pain', 'eye swelling', 'photophobia', 'hypoacusis', 'hearing impaired', 'visual impairment', 'visual acuity reduced', 'visual disturbance', 'visual field defect', 'visual brightness', 'visual brightness', 'visual disturbance', 'visual field defect', 'visual acuity reduced', 'visual impairment', 'visual disturbance'],
        'hematological symptoms': ['lymphadenopathy', 'full blood count', 'pallor', 'haemoglobin decreased', 'haematocrit decreased', 'red blood cell count decreased', 'white blood cell count decreased', 'platelet count decreased', 'blood iron decreased', 'blood iron increased', 'blood lactate dehydrogenase increased', 'blood bilirubin increased', 'blood creatinine increased', 'blood creatine phosphokinase increased', 'blood urea increased', 'blood uric acid increased', 'blood potassium increased', 'blood sodium decreased', 'blood sodium increased', 'blood chloride decreased', 'blood chloride increased', 'blood calcium decreased', 'blood calcium increased', 'blood albumin decreased', 'blood albumin increased', 'blood alkaline phosphatase increased', 'blood amylase increased', 'blood glucose increased', 'blood glucose decreased', 'blood triglycerides increased', 'blood cholesterol increased', 'blood cholesterol decreased', 'blood triglycerides decreased', 'blood triglycerides increased'],
        'urinary symptoms': ['urine analysis', 'urinary issues', 'urinary retention', 'urinary tract infection', 'urinary incontinence',],
        'swelling-related symptoms': ['swelling', 'swollen tongue', 'swelling face', 'peripheral swelling', 'peripheral coldness', 'peripheral circulatory failure', 'peripheral ischaemia', 'peripheral sensory neuropathy', 'peripheral motor neuropathy', 'peripheral vascular disorder',],
        'systemic infections': ['covid-19', 'influenza like illness', 'herpes zoster', 'cellulitis'],
        'allergic reactions': ['hypersensitivity', 'anaphylactic reaction', 'angioedema', 'anaphylactic shock', 'anaphylactoid reaction', 'anaphylactic transfusion reaction', 'anaphylactic response', 'anaphylactic symptom', 'anaphylactic reaction', 'anaphylactic shock', 'anaphylactoid reaction', ],
    }
    
    for category, symptoms in categories.items():
        if symptom.lower() in [s.lower() for s in symptoms]:
            return category
    
    return 'Other' 

# Function to categorize symptoms in a list and return groups
def categorize_symptom_list(symptom_list):
    categories = [categorize_symptoms(symptom) for symptom in symptom_list.split(', ')]
    return ', '.join(set(categories))  # Combine unique categories into a comma-separated string

moderna_df['Symptoms_category'] = moderna_df['Symptoms'].apply(categorize_symptom_list)


C:\Users\rutub\AppData\Local\Temp\ipykernel_13528\2969400966.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderna_df['Symptoms_category'] = moderna_df['Symptoms'].apply(categorize_symptom_list)


In [139]:
# Function to process the values in Symptoms_category column
def process_category(symptoms):
    symptoms_list = symptoms.split(', ')
    if 'Other' in symptoms_list and len(symptoms_list) > 1:
        symptoms_list.remove('Other')
        return ', '.join(symptoms_list)
    return symptoms

# Apply the function to the Symptoms_category column
moderna_df['Symptoms_category'] = moderna_df['Symptoms_category'].apply(process_category)

C:\Users\rutub\AppData\Local\Temp\ipykernel_13528\1114247338.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderna_df['Symptoms_category'] = moderna_df['Symptoms_category'].apply(process_category)


In [140]:
moderna_df.head()

,VAERS_ID,SEX,DIED,L_THREAT,HOSPITAL,DISABLE,OFC_VISIT,ER_ED_VISIT,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,Symptoms,AGE_GROUP,Symptoms_category
1538,905937,f,n,n,n,n,n,y,amlodipine 5 mg daily albuterol inhaler 2 puff...,not applicable,no concerns,aspirin - body rash codeine - anaphylaxis ceph...,moderna,"breath sounds abnormal, stridor, cough, throat...",ag6,"pain, skin reactions, respiratory symptoms"
1551,905962,f,n,n,n,n,n,y,no medication,not applicable,asthma - takes no medications adhd - takes no ...,vancomycin - rash,moderna,"chest discomfort, dyspnoea, heart rate increased",ag4,"pain, cardiovascular symptoms, respiratory sym..."
1565,905986,f,n,n,n,n,n,n,to many to list,not applicable,"b/p, diabetic, on a daily baby asa, chol meds,...","entex, pcn, adhesive tape sensitivity",moderna,"induration, mass, pruritus, skin warm, swelling",ag6,"skin reactions, swelling-related symptoms"
1653,906235,f,n,n,n,n,y,n,no medication,not applicable,no concerns,no allergies,moderna,"erythema, flushing, immediate post-injection r...",ag3,"skin reactions, neurological symptoms, cardiov..."
1697,906315,f,n,n,n,n,n,y,patient pre medicated self with benadryl per i...,not applicable,crohns,butalbital-aspirin-caffeine iodinated diagnost...,moderna,"ear swelling, erythema, lip swelling",ag7,skin reactions


In [141]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori

#this creates a new list for each entry by matching the dictionary with the entries 
def process_meds(row):
    meds_list = []
    
    if 'VAX_MANU' in row.index:
        vax_manu = row['VAX_MANU']
        if pd.notnull(vax_manu):
            meds_list.append(vax_manu)
    
    if 'Symptoms_category' in row.index:
        symptoms = row['Symptoms_category']
        if pd.notnull(symptoms):
            symptoms_list = symptoms.split(',')
            # symptoms_list = [symptom for symptom in symptoms_list if symptom != 'Other']
            meds_list.extend(symptoms_list)

    if 'AGE_GROUP' in row.index:
        age_group = row['AGE_GROUP']
        if pd.notnull(age_group):
            meds_list.append(age_group)
            
    return meds_list if meds_list else ['ignore']

moderna_df['APRIORI_LIST'] = moderna_df.apply(process_meds, axis=1)

# Step 2: Convert to List of Lists
transactions = moderna_df.groupby('VAERS_ID')['APRIORI_LIST'].apply(lambda x: [item for sublist in x for item in sublist]).tolist()
# for t in transactions:
#     print(t)

C:\Users\rutub\AppData\Local\Temp\ipykernel_13528\1721534185.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderna_df['APRIORI_LIST'] = moderna_df.apply(process_meds, axis=1)


In [142]:
moderna_df.head()

,VAERS_ID,SEX,DIED,L_THREAT,HOSPITAL,DISABLE,OFC_VISIT,ER_ED_VISIT,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,Symptoms,AGE_GROUP,Symptoms_category,APRIORI_LIST
1538,905937,f,n,n,n,n,n,y,amlodipine 5 mg daily albuterol inhaler 2 puff...,not applicable,no concerns,aspirin - body rash codeine - anaphylaxis ceph...,moderna,"breath sounds abnormal, stridor, cough, throat...",ag6,"pain, skin reactions, respiratory symptoms","[moderna, pain, skin reactions, respiratory ..."
1551,905962,f,n,n,n,n,n,y,no medication,not applicable,asthma - takes no medications adhd - takes no ...,vancomycin - rash,moderna,"chest discomfort, dyspnoea, heart rate increased",ag4,"pain, cardiovascular symptoms, respiratory sym...","[moderna, pain, cardiovascular symptoms, res..."
1565,905986,f,n,n,n,n,n,n,to many to list,not applicable,"b/p, diabetic, on a daily baby asa, chol meds,...","entex, pcn, adhesive tape sensitivity",moderna,"induration, mass, pruritus, skin warm, swelling",ag6,"skin reactions, swelling-related symptoms","[moderna, skin reactions, swelling-related sy..."
1653,906235,f,n,n,n,n,y,n,no medication,not applicable,no concerns,no allergies,moderna,"erythema, flushing, immediate post-injection r...",ag3,"skin reactions, neurological symptoms, cardiov...","[moderna, skin reactions, neurological sympto..."
1697,906315,f,n,n,n,n,n,y,patient pre medicated self with benadryl per i...,not applicable,crohns,butalbital-aspirin-caffeine iodinated diagnost...,moderna,"ear swelling, erythema, lip swelling",ag7,skin reactions,"[moderna, skin reactions, ag7]"


In [143]:
results = list(apriori(transactions, min_support=0.1, min_confidence=0.1, min_length=2, max_length=6))

# Print the results
for item in results:
    pair = item[0]
    if(len(pair)>1):
        print(item)
        items = [x for x in pair]
        print("Rule: " + items[0] + " -> " + items[1])
        
interesting_rules = []
for item in results:
    pair = item[0]
    if len(pair) > 1:
        items = [x for x in pair]
        antecedent = items[0]
        consequent = items[1]
        if antecedent == 'moderna' or consequent == 'moderna':
            interesting_rules.append(item)

# Print the interesting rules
for item in interesting_rules:
    pair = item[0]
    support = item[1]
    items = [x for x in pair]
    antecedent = items[0]
    consequent = items[1]
    print("Rule: " + antecedent + " -> " + consequent)
    print("Support: " + str(support))


RelationRecord(items=frozenset({'moderna', ' fatigue/general discomfort'}), support=0.155706177837491, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'moderna', ' fatigue/general discomfort'}), confidence=0.155706177837491, lift=1.0), OrderedStatistic(items_base=frozenset({' fatigue/general discomfort'}), items_add=frozenset({'moderna'}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({'moderna'}), items_add=frozenset({' fatigue/general discomfort'}), confidence=0.155706177837491, lift=1.0)])
Rule: moderna ->  fatigue/general discomfort
RelationRecord(items=frozenset({' fever/chills', 'moderna'}), support=0.2661961295903066, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' fever/chills', 'moderna'}), confidence=0.2661961295903066, lift=1.0), OrderedStatistic(items_base=frozenset({' fever/chills'}), items_add=frozenset({'moderna'}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({'m